In [1]:
from sklearn.datasets import fetch_covtype
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# 1. 데이터 
datasets = (fetch_covtype())

x = pd.DataFrame(datasets["data"])
y = pd.DataFrame(datasets["target"])


In [2]:
print(x.shape,y.shape) 

print(np.unique(y)) # [1 2 3 4 5 6 7]
# 불균형확인
# 참고로 이 y 값은 0 클래스가 없기 대문에 0 을 지워줘야합니다 ... ㅂㄷㅂㄷ

print(np.unique(y,return_counts=True)) # [211840, 283301,  35754,   2747,   9493,  17367,  20510



(581012, 54) (581012, 1)
[1 2 3 4 5 6 7]
(array([1, 2, 3, 4, 5, 6, 7]), array([211840, 283301,  35754,   2747,   9493,  17367,  20510],
      dtype=int64))


In [3]:
# print(x.info())
# print(x.describe())
# 데이터 분리
x_train, x_test, y_train,y_test = train_test_split(
                                                    x,
                                                    y,
                                                    train_size=0.8,
                                                    
                                                    shuffle = True,
                                                    #  stratify 는 데이터 불균형을 해결해줌
                                                    stratify=y,
                                                    random_state=21
                                                   )
# print(x.columns)
# y 를 원 핫 인코딩 변환
y_train = tf.one_hot(y_train[0],depth =8)

# y 의 0 번 칼럼을 드랍해야함 왜 why ? 원핫 인코딩한뒤 자동으로 0 칼럼이 붇기 때문
y_train = pd.DataFrame(y_train).drop(columns=[0])

y_test = tf.one_hot(y_test[0],depth= 8)
y_test = pd.DataFrame(y_test).drop(columns=[0])


print(y_train[:10])
print("y_train shape : ",y_train.shape)
print("y_test shape : ",y_test.shape)

     1    2    3    4    5    6    7
0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
1  1.0  0.0  0.0  0.0  0.0  0.0  0.0
2  1.0  0.0  0.0  0.0  0.0  0.0  0.0
3  0.0  1.0  0.0  0.0  0.0  0.0  0.0
4  1.0  0.0  0.0  0.0  0.0  0.0  0.0
5  0.0  1.0  0.0  0.0  0.0  0.0  0.0
6  0.0  1.0  0.0  0.0  0.0  0.0  0.0
7  0.0  1.0  0.0  0.0  0.0  0.0  0.0
8  1.0  0.0  0.0  0.0  0.0  0.0  0.0
9  1.0  0.0  0.0  0.0  0.0  0.0  0.0
y_train shape :  (464809, 7)
y_test shape :  (116203, 7)


In [4]:
model = Sequential([
    Dense(128,activation="relu",input_shape=(54,)),
    Dense(128,activation="relu") ,
    Dense(128,activation="relu") ,
    Dense(64,activation="relu") ,
    Dense(64,activation="relu") ,
    Dense(32,activation="relu") ,
    Dense(32,activation="relu") ,
    # 다중분류모델의 활성화 함수는 softmax 입니다
    Dense(7,activation="softmax")
])

In [5]:
# 이진분류는 마지막 활성함수는 sigmoid + loss 는 바이너리 크로스 엔트로피 
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
monitor='val_loss',
patience=10, 
verbose=1, 
mode='min')

# 훈련값이 int 형이기 때문에 sparse 를 사용합니다.
model.compile(loss="categorical_crossentropy"
              ,optimizer="adam"
              ,metrics=["accuracy"]
              )

hist = model.fit(x_train, y_train, epochs=100, 
                 validation_split=0.2,batch_size=256,
                 callbacks = [early_stopping])

# metrics 에 accuracy 사용가능


Epoch 1/100
1453/1453 [==============================] - 9s 5ms/step - loss: 1.2299 - accuracy: 0.6000 - val_loss: 0.7677 - val_accuracy: 0.6701
Epoch 2/100
1453/1453 [==============================] - 8s 6ms/step - loss: 0.7035 - accuracy: 0.6981 - val_loss: 0.6426 - val_accuracy: 0.7227
Epoch 3/100
 713/1453 [=============>................] - ETA: 3s - loss: 0.6490 - accuracy: 0.7193

KeyboardInterrupt: 

In [ ]:
loss,accuracy = model.evaluate(x_test,y_test)

print("loss:",loss,"\nacc :" ,accuracy)


3632/3632 [==============================] - 7s 2ms/step - loss: 0.2510 - accuracy: 0.9001
loss: 0.25100550055503845 
acc : 0.9000627994537354


In [ ]:
from sklearn.metrics import accuracy_score

y_predict = model.predict(x_test)
print(y_predict)
y_predict = np.argmax(y_predict,axis=1)


[[9.9999928e-01 4.6706285e-07 0.0000000e+00 ... 5.2797424e-14
  0.0000000e+00 2.3549717e-07]
 [2.3933532e-04 9.9971741e-01 2.2561000e-05 ... 2.0765980e-05
  3.5783958e-09 6.0112543e-35]
 [9.9874628e-01 6.9678295e-04 0.0000000e+00 ... 1.3404945e-10
  0.0000000e+00 5.5694004e-04]
 ...
 [1.6505672e-02 9.8347962e-01 9.4122252e-06 ... 9.1939233e-07
  4.4419007e-06 1.6440261e-09]
 [3.8507733e-01 3.9241138e-01 4.8341703e-06 ... 2.2248405e-01
  2.0870464e-05 1.5434058e-06]
 [1.3462229e-01 8.6527860e-01 7.6926772e-06 ... 9.0807531e-05
  2.3793865e-07 2.9445823e-07]]


In [ ]:
print(y_predict)

y_predict = tf.one_hot(y_predict,depth =7)


[0 1 0 ... 1 1 1]


In [ ]:
# 데이터 출력

print("예측데이터 : ",y_predict[:20])
print("실제데이터 : ",y_test[:20])

print
print("========================================")
acc = accuracy_score(y_test,y_predict)
print(acc)


예측데이터 :  tf.Tensor(
[[1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]], shape=(20, 7), dtype=float32)
실제데이터 :        1    2    3    4    5    6    7
0   1.0  0.0  0.0  0.0  0.0  0.0  0.0
1   0.0  1.0  0.0  0.0  0.0  0.0  0.0
2   1.0  0.0  0.0  0.0  0.0  0.0  0.0
3   1.0  0.0  0.0  0.0  0.0  0.0  0.0
4   0.0  1.0  0.0  0.0  0.0  0.0  0.0
5   1.0  0.0  0.0  0.0  0.0  0.0  0.0
6   1.0  0.0  0.0  0.0  0.0  0.0  0.0
7   1.0  0.0  0.0  0.0  0.0  0.0  0.0
8   0.0  1.0  0.0  0.0  0.0  0.0  0.0
9   1.0  0.0  0.0  0.0  0.0  0.0  0.0
10  0.0  1.0  0.0  0.0  0.0  0.0  0.0
11 

In [ ]:
model.save("./_data\\"+"fetch(val_acc=0.89).h5")